Identify popular feature layers (based on item count), recently updated layers, and feature layers that use a lot of database storage

In [ ]:
import datetime as dt
from pytz import timezone
from dateutil.relativedelta import relativedelta

from arcgis.gis import GIS

In [ ]:
gis = GIS(profile="your_gis_admin", verify_cert=False)

## Popular feature layers based on number of views

### Create datetime objects to represent time ranges

In [ ]:
# represent local timezone information
pst_tz = timezone("America/Los_Angeles")

In [ ]:
# construct datetime objects for time range queries
mar19 = dt.datetime.now()
six_mos = relativedelta(months=6)
sep19 = mar19 - six_mos

# construct millisecond timestamps for searching
mar19_qy = int(mar19.timestamp()*1000)
sep19_qy = int(sep19.timestamp()*1000)

In [ ]:
fl_6mos = gis.content.search(query=f"created: [{sep19_qy} TO {mar19_qy}]",
                             item_type="Feature Layer",
                             sort_field="numViews",
                             sort_order="desc",
                             max_items=500)
len(fl_6mos)

In [ ]:
for flyr in fl_6mos:
    print(f"{flyr.title[:56]:58}{flyr.numViews}")

## Recently updated feature layers

In [ ]:
# relative date objects
twenty_four_hrs = relativedelta(hours=24)
two_wks = relativedelta(weeks=2)
one_mo = relativedelta(months=1)
two_mo = relativedelta(months=2)

deltas = [twenty_four_hrs, two_wks, one_mo, two_mo]

# datetime objects 
hrs6_prev = mar19 - twenty_four_hrs
wks2_prev = mar19 - two_wks
mos1_prev = mar19 - one_mo
mos2_prev = mar19 - two_mo

time_pd_objs = [mar19 - rd for rd in deltas]

# corresponding millisecond values for platform queries
hrs6_prev_qy = int(hrs6_prev.timestamp()*1000)
wks2_prev_qy = int(wks2_prev.timestamp()*1000)
mos1_prev_qy = int(mos1_prev.timestamp()*1000)
mos2_prev_qy = int(mos2_prev.timestamp()*1000)

query_objs = [int(time_pd.timestamp()*1000) for time_pd in time_pd_objs]

In [ ]:
time_ranges = ["day ago", "two weeks ago", "one month ago", "two_months_ago"]

In [ ]:
for readable, date_ms in list(zip(time_ranges, query_objs)):
    print(f"{readable}\n{'-'*50}")
    flyr_list = gis.content.search(query=f"modified: [{date_ms} TO {mar19_qy}]",
                                   max_items=100,
                                   sort_field="type",
                                   sort_order="asc")
    print(f"{' ':3}{len(flyr_list)} items(s) modified\n{' '*3}{'='*20}")
    for flyr in flyr_list:
        print(f"{' ':5}{flyr.title[:38]:40}{flyr.type:30}{flyr.size}")
    print("\n")

## Feature layers using above 1MB of database storage

In [ ]:
mod_fl_mos2 = [fl 
               for fl in gis.content.search(query=f"modified: [{mos2_prev_qy} TO {mar19_qy}]",
                                            max_items=100)
               if fl.size/(1024*1024) > 1]

In [ ]:
def get_mb(item):
    return item.size/(1024*1024)

In [ ]:
mod_fl_mos2.sort(key=get_mb, reverse=True)

In [ ]:
for item in mod_fl_mos2:
    print(f"{item.title[:35]:40}{item.type:20}{item.size/(1024 * 1024):.3} MB")

## Credit Usage

In [ ]:
week_prev = mar19 - relativedelta(weeks=1)

In [ ]:
credit_mgr = gis.admin.credits
credit_usg_week = credit_mgr.credit_usage(mar19, week_prev)

In [ ]:
credit_usg_week

In [ ]:
total = sum(credit_usg_week.values())
total

In [ ]:
credit_pct = {k:(v/total*100) for k, v in credit_usg_week.items()}
credit_pct

### User Usage Reports

In [ ]:
usage_reports = gis.admin.usage_reports

In [ ]:
user_usage = usage_reports.users(start_time=mar19, time_frame="60days")

In [ ]:
def get_users_consuming_credits():
    usage_totals = []
    for usg_dict in user_usage['data']:
        if 'username' in usg_dict:
            total_usg = sum([float(ud[1]) for ud in usg_dict['credits']])
            if total_usg > 0.0:
                #print(f"{usg_dict['username']}\n{'-'*50}")
                #print(f"\t...used {total_usg} credits during this time")
                usage_totals.append({usg_dict['username']:total_usg})
    return usage_totals

In [ ]:
user_credit_use = get_users_consuming_credits()
for usage_stats in user_credit_use:
    for user, credits in usage_stats.items():
        print(f"{user:40}{credits:>5.4}")